# Plotting Spectra 
This python notebook is going to outline how to load a fits file containing a spectra and preform basic manipulation and fitting of the data.


In [ ]:
#We will need several notebooks
import matplotlib.pyplot as plt #used to plot the spectra
import numpy as np #very useful for manipulating arrays
from astropy.io import fits #used to import fits files
from scipy.special import wofz #needed specifically for the voigt function
from scipy.optimize import curve_fit #a pretty good fitting function within the scipy module

**Flexible Image Transport System (FITS)** is an open standard defining a digital file format useful for storage, transmission and processing of data: formatted as multi-dimensional arrays (for example a 2D image), or tables.

This is the standard way in which images and spectra are stored in astronomy. The FITS file format has a header which contains useful information about the data including when the observation occured, what time the observation occured, what telescope was used for the observation, ect. The second part is the data. The data can be in many formats including a simple 2D array (an image) or a cube. 

In [ ]:
#Our first step will be to open the FITS file. You were provided a spectrum of the Sun. The file is titled 884752_melchiors_spectrum.fits
# The data is from Melchiors which is a library of science ready high-resolution spectra. https://www.royer.se/melchiors.html
hdul = fits.open('884752_melchiors_spectrum.fits')
#this will print the information that is displayed in the FITS file.
hdul.info()

In [ ]:
#next you need to investigate the headers so you can learn about what format the spectra is being presented in.
#execute the next line. It will print the entire header in the first hdu
#note that python uses zero indexing meaning the first instance of something is zero not 1
print(repr(hdul[0].header))

In [ ]:
#Lets now look at the next header which has even more information
#this header is associated with the data as the data for this FITS file is storred in the second HDU 
#. (which is 1 with python zero indexing)

#note that there are 7 rows of information, TTYPE# is saying what is in each element of the list. 
#. We want to use the normalized flux
print(repr(hdul[1].header))

In [ ]:
#the python notebook we are using through a browser doesn't like the spectral data (unsure why)
# so I saved the data in a csv file for now.
"""
spectrum_list = hdul[1].data #import all of the data
#for some reason they put the data in the format of a numpy array that contians lists. Which is very difficult to plot
# there is almost certainly a way to plot this better, but this is what I did.
#create a numpy array that is three by the length of the spectrum
norm_flux = np.zeros((3,len(spectrum_list)))

#do a for loop over the length of the spectrum that puts the wavelength, normalized spectrum, and uncertainty into
# a useful numpy array
for i in range(0,len(spectrum_list)):
    norm_flux[0,i] = spectrum_list[i][1]
    norm_flux[1,i] = spectrum_list[i][2]
    norm_flux[2,i] = spectrum_list[i][3]
"""
#Use this part of the code
norm_flux = np.genfromtxt("sun_normalized_flux.csv", delimiter=',')

In [ ]:
#plot the entire spectrum
#there are lots of ways to make plots. here is a simple way to do so
fig, ax = plt.subplots(figsize=(20, 5)) #make the image wide to make it easier to see the spectrum
plt.plot(norm_flux[0],norm_flux[1],'-',color='k') #plot x, y, line type, line color, 
plt.xlabel('Wavelength (Ang)')
plt.ylabel('Normalized Flux')
plt.show()
plt.close()
#important to close your figures or it will be hanging out in memory for a while

In [ ]:
#plot around H-alpha
halpha_lam = 6562.81 #halpha 

#I don't care about the figure size this time, so I didn't declare the figure like I did above
plt.plot(norm_flux[0],norm_flux[1],'-',color='k')
plt.title('ADD TITLE HERE')
plt.xlabel('Wavelength (Ang)')
plt.ylabel('Normalized Flux')
plt.xlim(halpha_lam - 25, halpha_lam + 25) #I can specify how much of the spectrum I want to plot
plt.plot((halpha_lam, halpha_lam), (0.1,0.15),'-',color='r',lw=2) #draw a read line where H-alpha is located
#plt.savefig('Ni_line_' + str(halpha_lam) + '_Ang.png', dpi=300)
plt.show()
plt.close()

# Question 1

Modifying the cell above, plot the following lines. 
Make sure to mark where the line is, and add a title to the top of the figure
save the figures as a png.

identify the following lines and create plots for them

Hbeta 4861.34 ang

Calcium H line (Ca+) = 3968.47 ang

Calcium K line (Ca+) = 3933.66 ang

Fe = 5270.39 ang

In [ ]:
#Use this cell for question #1, you can add more cells as necessary. 



# Question 2

Explore the sun's spectrum. Are there any features that surprise you? Do all of the major lines we noted above look the same? Are there any features of the spectrum that are possibly dominated by reduction processes? Answer in the cell below.



# Fitting line profiles

Our ultimate goal is to fit line profiles to the spectrum. We will first define the functions that we will need. Then we will use curve.fit to fit the functions to the spectrum.

In [ ]:
#Our ultimate goal is to measure the abundances of a star. We are going to start with something like the Sun.
#Our first step is to measure the EW of the spectral lines.

#define the functions that we might need

#You should define functions for anything you might use repeditavly.

#Voigt function
def voigt(x, x0, A, sigma, gamma):
    """
    Computes the Voigt line profile.

    Parameters
    ----------
    x : array_like
        The independent variable.
    sigma : float
        The standard deviation of the Gaussian component.
    gamma : float
        The HWHM of the Lorentzian component.

    Returns
    -------
    array_like
        The Voigt profile.
    """
    z = (((x-x0) + 1j * gamma) / (sigma * np.sqrt(2)))
    return 1 - A*np.real(wofz(z)) / (sigma * np.sqrt(2 * np.pi))

#Gaussian Function
def gaussian (x, x0, A, sigma):
    return 1 - A*np.exp(-(x-x0)**2/(2*sigma**2))

#Lorentzian function
def lorentzian (x, x0, A, gamma):
    return 1 - A*gamma**2/(np.pi*gamma**2 + ((x-x0)**2))

#gaussian area
def gaussian_area(A,sigma):
    return A*np.sqrt(np.pi*2)*sigma

In [ ]:
line = 6597.56 #halpha; wavelength of the line in angstroms
width = 10 #width of area you want to fit. Not in angstroms but in the spectral resolution (think pixels)

a = np.argmin(np.abs(norm_flux[0,:]-line)) #finding where the line is located in index space

# prompt: fit gaussian to data. We are going to use curve fit. We provide it with:
# -the function, we defined above (gaussian)
# - the wavelength
# - the flux,
# - the uncertainty in the flux (sigma)
# - an estimate of the initail conditions (p0)
# - and the bounds
#The output is the best fit values (popt) and the covariance matrix (pcov)
popt, pcov = curve_fit(gaussian, norm_flux[0,a-width:a+width], 
                       norm_flux[1,a-width:a+width], sigma=norm_flux[2,a-width:a+width], 
                       p0=[line, 0.4, 2], bounds=(0,[np.inf,1.0,np.inf]))

#taking the diagonal of the function and the square provides you with a
#   1-sigma estimate of the uncertainties for each fit parameter
perr = np.sqrt(np.diag(pcov)) 

print('Gaussian estimate of the line: ', popt)
print('and associated uncertainty:', perr)

#based on the best fit parameters, calculate what the fit line profile would be
output = gaussian (norm_flux[0,a-width:a+width], popt[0], popt[1], popt[2])

#Finally calculate the area of the gaussian using the best fit parameters
# we should probably propogate uncertainty here but we will save that for another day
print('Gaussian EW (ang): ',gaussian_area(popt[1],popt[2]))

plt.plot(norm_flux[0], norm_flux[1], '-', color='k') #the origional spectrum
plt.plot(norm_flux[0,a-width:a+width], output, '--', color='r') #the result of the gaussian fit
#plt.plot(norm_flux[0,a-width:a+width], l_output, '--', color='g')
plt.plot((halpha_lam, halpha_lam), (0.1,0.15),'-',color='r',lw=2) #draw a read line where H-alpha is located
plt.xlabel('Wavelength (Ang)')
plt.ylabel('Normalized Flux')
plt.title('Fe line ' + str(line) + ' Ang')
plt.xlim(line-2,line+2)
#plt.ylim(0.3,1.05)
plt.savefig('Fe_line_' + str(line) + '_Ang.png', dpi=300) #saving the figure with 300 dots per inch (resolution)
plt.show()
plt.close()

# Question 3

Modify the above code (or copy it into a new cell) and fit a lorentzian function to the line.

Question: Does the Lorentzian line fit better or worse than the gaussian? Type your answer in the cell below.



# Question 4

Our next goal is to measure the equivalent widths for more lines.

Below is a list of potentialy "well behaved" lines that have previously been used for successful abundance work.

Questions that you want to consider:
1) How do you plan to record and store your results?
2) How to you plan to verify that each line you measure has a good fit?
3) Do you plan to test if each line is best fit by a gaussian?

Answer in the cell below

6597.56 Fe

6627.54 Fe

6646.93 Fe 

6653.91 Fe 

6677.99 Fe 

6703.57 Fe 

6710.32 Fe 

6725.36 Fe 

6726.67 Fe 

6733.15 Fe 

6806.86 Fe 

6810.27 Fe 

6820.37 Fe 

6717.68 Ca 

6721.85 Si 

6643.63 Ni 

6767.77 Ni 

6772.31 Ni

In [ ]:
#if you have time, start measuring the EW of the lines noted above.